In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import backend as K
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os

from google.colab import drive
drive.mount('/content/drive')

# Set up constants
IMG_SIZE = (224, 224)  # MobileNetV2 default input size
BATCH_SIZE = 32
EPOCHS = 30

# Define the path to the images folder
data_dir = '/content/drive/My Drive/images'
print("Contents of data_dir:")
print(os.listdir(data_dir))



Mounted at /content/drive
Contents of data_dir:
['NV', 'MEL', 'BCC', 'SCC']


In [ ]:
# Update class names to include SCC
class_names = ['MEL', 'NV', 'BCC', 'SCC']
for class_name in class_names:
    if not os.path.isdir(os.path.join(data_dir, class_name)):
        raise ValueError(f"Folder {class_name} not found in {data_dir}")

# Set up data generators with increased augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    classes=class_names,
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    classes=class_names,
    shuffle=True
)



Found 7190 images belonging to 4 classes.
Found 1795 images belonging to 4 classes.


In [ ]:
# Load pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))

# Fine-tune the model
for layer in base_model.layers:
    layer.trainable = True

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(4, activation='softmax')(x)  # 4 classes

# Create the final model
model = Model(inputs=base_model.input, outputs=output)



9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Define F1 Score metric as a class
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# Compile the model with additional metrics
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall(), F1Score()])



In [ ]:
# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weight_dict = dict(enumerate(class_weights))

# Train the model with class weights
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    class_weight=class_weight_dict
)

# Save the model
model.save('/content/drive/My Drive/models/skin_lesion_classifier_mobilenetv2.h5')



Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


224/224 ━━━━━━━━━━━━━━━━━━━━ 3468s 15s/step - accuracy: 0.5436 - f1_score: 0.4759 - loss: 1.1110 - precision: 0.6288 - recall: 0.3871 - val_accuracy: 0.7500 - val_f1_score: 0.7502 - val_loss: 1.4346 - val_precision: 0.7504 - val_recall: 0.7500
Epoch 2/30
  1/224 ━━━━━━━━━━━━━━━━━━━━ 31s 143ms/step - accuracy: 0.8438 - f1_score: 0.7797 - loss: 0.4789 - precision: 0.8519 - recall: 0.7188

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


224/224 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.8438 - f1_score: 0.7797 - loss: 0.4789 - precision: 0.8519 - recall: 0.7188 - val_accuracy: 0.6667 - val_f1_score: 0.6667 - val_loss: 1.4237 - val_precision: 0.6667 - val_recall: 0.6667
Epoch 3/30
224/224 ━━━━━━━━━━━━━━━━━━━━ 295s 1s/step - accuracy: 0.7319 - f1_score: 0.7276 - loss: 0.6001 - precision: 0.7711 - recall: 0.6889 - val_accuracy: 0.7500 - val_f1_score: 0.7502 - val_loss: 1.6652 - val_precision: 0.7504 - val_recall: 0.7500
Epoch 4/30
224/224 ━━━━━━━━━━━━━━━━━━━━ 0s 234us/step - accuracy: 0.8125 - f1_score: 0.7333 - loss: 0.3536 - precision: 0.7857 - recall: 0.6875 - val_accuracy: 0.3333 - val_f1_score: 0.3333 - val_loss: 3.9664 - val_precision: 0.3333 - val_recall: 0.3333
Epoch 5/30
224/224 ━━━━━━━━━━━━━━━━━━━━ 250s 985ms/step - accuracy: 0.7596 - f1_score: 0.7578 - loss: 0.5026 - precision: 0.7886 - recall: 0.7294 - val_accuracy: 0.7522 - val_f1_score: 0.7522 - val_loss: 1.7336 - val_precision: 0.7522 - val_recall: 0.

In [ ]:
# Print class indices
# print("Class indices:", train_generator.class_indices)
model_saved = tf.keras.models.load_model('/content/drive/My Drive/models/skin_lesion_classifier_mobilenetv2.h5')
# Update class names to include SCC
class_names = ['MEL', 'NV', 'BCC', 'SCC']
for class_name in class_names:
    if not os.path.isdir(os.path.join(data_dir, class_name)):
        raise ValueError(f"Folder {class_name} not found in {data_dir}")

# Function to predict image
def predict_image(img_path, nv_threshold=0.7):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model_saved.predict(img_array)

    if np.argmax(prediction) == class_names.index('NV') and prediction[0][class_names.index('NV')] < nv_threshold:
        predicted_class = class_names[np.argsort(prediction[0])[-2]]
    else:
        predicted_class = class_names[np.argmax(prediction)]

    confidence = np.max(prediction)

    return predicted_class, confidence

finish_dir = '/content/drive/My Drive/finish'
# Interactive prediction loop


In [ ]:
while True:
    user_input = input("Enter an image number (1-1000) or 'q' to quit: ")

    if user_input.lower() == 'q':
        break

    try:
        image_number = int(user_input)

        for filename in os.listdir(finish_dir):
            if filename.startswith(f"{image_number}.") and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(finish_dir, filename)

                predicted_class, confidence = predict_image(img_path)

                print(f"Image: {filename}")
                print(f"Predicted class: {predicted_class}")
                print(f"Confidence: {confidence:.2f}")
                print()
                break
        else:
            print(f"No image found with number {image_number}")

    except ValueError:
        print("Invalid input. Please enter a number or 'q' to quit.")

print("Thank you for using the classifier!")

Enter an image number (1-1000) or 'q' to quit: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Image: 1.jpg
Predicted class: NV
Confidence: 0.74

Enter an image number (1-1000) or 'q' to quit: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Image: 2.jpg
Predicted class: NV
Confidence: 0.99



KeyboardInterrupt: Interrupted by user

In [ ]:
for i in range(1,54):
    user_input=i
    try:
        image_number = int(user_input)

        for filename in os.listdir(finish_dir):
            if filename.startswith(f"{image_number}.") and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(finish_dir, filename)

                predicted_class, confidence = predict_image(img_path)

                print(f"Image: {filename}")
                print(f"Predicted class: {predicted_class}")
                print(f"Confidence: {confidence:.2f}")
                print()
                break
        else:
            print(f"No image found with number {image_number}")

    except ValueError:
        print("Invalid input")

print("Thank you for using the classifier!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Image: 1.jpg
Predicted class: NV
Confidence: 0.74

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Image: 2.jpg
Predicted class: NV
Confidence: 0.99

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Image: 3.jpg
Predicted class: NV
Confidence: 1.00

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Image: 4.jpg
Predicted class: SCC
Confidence: 0.70

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Image: 5.jpg
Predicted class: NV
Confidence: 1.00

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Image: 6.jpg
Predicted class: NV
Confidence: 0.74

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Image: 7.jpg
Predicted class: MEL
Confidence: 0.81

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Image: 8.jpg
Predicted class: NV
Confidence: 1.00

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
Image: 9.jpg
Predicted class: NV
Confidence: 1.00

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Image: 10.jpg
Predicted class: NV
Confidence: 1.00

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Image: 11.jpg
Predicted class: NV
Confidence: 1.00

1/1 ━━━━━━━━